# ALM Pipeline Demo (Equality One-Hot)

This notebook demonstrates a **stateless** ALM outer loop using:
- `ALMAlgorithm` (stateless)
- `ConstraintRegistry` (holds user equalities/inequalities and lifted-discrete blocks)

Scenarios: <br>
- Continuous with equality and inequality
- Lifted binary block (sum-to-one + one-hot as equality)
- Multi-level block with decoding
- Inner solve with L-BFGS-B (scenario 1-3) and Dirac-3 continuous cloud solver (scenario 4)

In [1]:
import numpy as np
from eqc_models.base.polynomial import PolynomialModel
from eqc_models.algorithms import ALMAlgorithm, ConstraintRegistry, ALMConfig, ALMBlock
from eqc_models.solvers.qciclient import Dirac3ContinuousCloudSolver
from scipy.optimize import minimize

class LBFGSBSolver:
    def solve(self, poly_model: PolynomialModel, **kwargs):
        x0 = getattr(poly_model, "initial_guess", None)
        n = int(poly_model.n)
        lb = getattr(poly_model, "lower_bound", np.full(n, 0))
        ub = getattr(poly_model, "upper_bound", np.full(n, +np.inf))
        if x0 is None:
            if lb is not None and ub is not None:
                x0 = 0.5 * (np.asarray(lb, float) + np.asarray(ub, float))
            else:
                x0 = np.ones(n, dtype=float) / max(1, n)

        # reconstruct Q, c from terms (quadratic + linear only)
        Q = np.zeros((n, n), float); c = np.zeros(n, float)
        for inds, coeff in zip(poly_model.polynomial.indices, poly_model.polynomial.coefficients):
            if len(inds) == 2 and inds[0] == 0:
                c[inds[1]-1] += coeff
            elif len(inds) == 2 and inds[0] != 0:
                i, j = inds
                if i == j:
                    Q[i-1, i-1] += coeff
                else:
                    Q[i-1, j-1] += coeff/2.0
                    Q[j-1, i-1] += coeff/2.0

        Qs = 0.5*(Q + Q.T)
        def f(x): return float(x @ (Qs @ x) + c @ x)
        def g(x): return Qs @ x + c
        bounds = None
        if lb is not None and ub is not None:
            bounds = [(float(lb[i]), float(ub[i])) for i in range(n)]
        res = minimize(f, x0, jac=g, method="L-BFGS-B", bounds=bounds,
                       options={"maxiter": 2000, "ftol": 1e-12})
        return {"x": np.asarray(res.x, float), "success": bool(res.success), "fun": float(res.fun)}

def poly_builder_quadratic(Q, c):
    """
    Convert Q, c into polynomial monomials compatible with ALMAlgorithm._poly_value.
    We keep only linear/quadratic monomials in tests.
    """
    terms = []
    Qs = 0.5*(Q + Q.T)
    n = Q.shape[0]
    for i in range(n):
        if Qs[i, i] != 0.0:
            terms.append(((i + 1, i + 1), float(Qs[i, i])))
        for j in range(i+1, n):
            q = 2.0 * Qs[i, j]
            if q != 0.0:
                terms.append(((i + 1, j + 1), float(q)))
    for i in range(n):
        if c[i] != 0.0:
            terms.append(((0, i + 1), float(c[i])))

    return terms

## Scenario 1: Continuous with equality & inequality

$\mathrm{min} \; 0.5*||\vec{x}||^2$ s.t. $\vec{1}^T \vec{x} = 1$, $\forall x_i \in \vec{x}, x_i >= 0$, and $x_1 + x_2 - 0.8 <= 0$

In [2]:
n = 3
Q = 0.5 * np.eye(n)
c = np.zeros(n)

terms = poly_builder_quadratic(Q, c)

model = PolynomialModel([w for (_, w) in terms], [inds for (inds, _) in terms])
model.upper_bound = np.ones(n) * 10

reg = ConstraintRegistry()
Aeq = np.ones((1, n))
beq = np.array([1.0])
reg.add_equality(fun=lambda x, A=Aeq, b=beq: A@x - b,
                 jac=lambda x, A=Aeq: A, name="sum1")

Gineq = np.zeros((1, n))
Gineq[0, 0] = 1.0
Gineq[0, 1] = 1.0
hineq = np.array([0.8])
reg.add_inequality(fun=lambda x, G=Gineq, h=hineq: G@x - h, 
                   jac=lambda x, G=Gineq: G, name="cap12")

cfg = ALMConfig(max_outer=60, tol_h=1e-8, tol_g=1e-8, rho_h=10.0, rho_g=10.0)
x0 = np.ones(n) / n

res1 = ALMAlgorithm.run(model, reg, LBFGSBSolver(), cfg, x0=x0, verbose=True)
print('x* =', res1['x'])
print('eq_inf =', res1['hist']['eq_inf'][-1], 'ineq_inf =', res1['hist']['ineq_inf'][-1])

[ALM 00] f=0.166667 | eq_inf=0.00e+00 | ineq_inf=0.00e+00 | rho_h=1.00e+01 | rho_g=1.00e+01
[ALM] converged at iter 0
x* = [0.33333333 0.33333333 0.33333333]
eq_inf = 0.0 ineq_inf = 0.0


## Scenario 2: Lifted binary block (sum-to-one + equality one-hot)

In [10]:
n = 2
terms = [((0, 2), -1.0)] # favor second component
model2 = PolynomialModel([w for (_, w) in terms], [inds for (inds, _) in terms])
model2.lower_bound = np.zeros(n)
model2.upper_bound = np.ones(n)

reg2 = ConstraintRegistry()
levels = np.array([0.0, 1.0])
reg2.add_block(idx=range(0, 2), levels=levels, sum_to_one=True, one_hot=True)

x0 = np.array([0.5, 0.5])
cfg2 = ALMConfig(max_outer=80, tol_h=1e-8, tol_g=1e-8, rho_h=20.0, rho_g=20.0, adapt=False)

res2 = ALMAlgorithm.run(model2, reg2, LBFGSBSolver(), cfg2, x0=x0, verbose=True)
x2 = res2["x"]
print("s* =", x2, "  sum(s) =", x2.sum())
M2 = np.ones((n,n)) - np.eye(n)
print("one-hot residual:", float(x2 @ (M2 @ x2)))
print("decoded =", res2["decoded"])

[ALM 00] f=-0.5 | eq_inf=5.00e-01 | ineq_inf=0.00e+00 | rho_h=2.00e+01 | rho_g=2.00e+01
[ALM 01] f=-0.5 | eq_inf=5.00e-01 | ineq_inf=0.00e+00 | rho_h=2.00e+01 | rho_g=2.00e+01
[ALM 02] f=-0.525 | eq_inf=4.75e-01 | ineq_inf=0.00e+00 | rho_h=2.00e+01 | rho_g=2.00e+01
[ALM 03] f=-1 | eq_inf=0.00e+00 | ineq_inf=0.00e+00 | rho_h=2.00e+01 | rho_g=2.00e+01
[ALM] converged at iter 3
s* = [0. 1.]   sum(s) = 1.0
one-hot residual: 0.0
decoded = {0: 1.0}


## Scenario 3: Multi-level block (equality one-hot)

One 3-level variable with levels [0, 2, 5]; objective favors level 5

In [6]:
n = 3
terms = [((0, 3), -2.0)]
model3 = PolynomialModel([w for (_, w) in terms], [inds for (inds, _) in terms])
model3.upper_bound = np.ones(n)

reg3 = ConstraintRegistry()
levels3 = np.array([0.0, 2.0, 5.0])
reg3.add_block(idx=range(0, 3), levels=levels3, sum_to_one=True, one_hot=True)

x0 = np.ones(n)/n
cfg3 = ALMConfig(max_outer=60, tol_h=1e-6, tol_g=1e-6, rho_h=30.0, rho_g=30.0)

out3 = ALMAlgorithm.run(model3, reg3, LBFGSBSolver(), cfg3, x0=x0, verbose=True)
x3 = out3["x"]
decoded = out3["decoded"]
print("s* =", x3, "  sum(s*)=", x3.sum())
M3 = np.ones((n,n)) - np.eye(n)
print("one-hot residual:", float(x3 @ (M3 @ x3)))
print("decoded:", decoded)

[ALM 00] f=-0.666667 | eq_inf=6.67e-01 | ineq_inf=0.00e+00 | rho_h=3.00e+01 | rho_g=3.00e+01
[ALM 01] f=-1.472 | eq_inf=2.64e-01 | ineq_inf=0.00e+00 | rho_h=3.00e+01 | rho_g=3.00e+01
[ALM 02] f=-2 | eq_inf=0.00e+00 | ineq_inf=0.00e+00 | rho_h=3.00e+01 | rho_g=3.00e+01
[ALM] converged at iter 2
s* = [0. 0. 1.]   sum(s*)= 1.0
one-hot residual: 0.0
decoded: {0: 5.0}


## Scenario 4: Multi-block lifted one-hot variables with ALM + Dirac-3 inner solver
- No continuous variables outside the lifted blocks
- Global device sum constraint required by Dirac-3 equals the number of blocks

Requirements:
- `Dirac3ContinuousCloudSolver` from `eqc-models`
- Setting of `API_TOKEN` and `API_URL` for Dirac-3 access as environment variables
- `PolynomialModel` from `eqc-models`

In [7]:
import os

os.environ["QCI_TOKEN"] = "YOUR_API_TOKEN"
os.environ["QCI_API_URL"] = "https://api.qci-prod.com/"

In [9]:
# ---- define blocks directly (levels = allowed valuess for each discrete var) ----
blocks = [
    np.array([0.0, 1.0]),           # Block 0: binary (size 2)
    np.array([0.0, 2.0, 5.0])       # Block 1: 3 -level (size 3)
]

# build the flattened dimension and remember slices for each block
slices = []
n = 0
for levels in blocks:
    k = len(levels)
    slices.append(slice(n, n + k))
    n += k

# ---- objective: simple linear tilt to prefer a specific level in each block ----
# choose the preferred index in each block (0-based)
preferred = [1, 2]
terms = []
for (sl, pref) in zip(slices, preferred):
    terms.append(((0, sl.start + pref + 1), -1.0))

# ---- Model ----
model4 = PolynomialModel([w for (_, w) in terms], [inds for (inds, _) in terms])
model4.upper_bound = np.ones(n)

# register each block; passed to ALM run for install:
#   - sum-to-one equality: sum(s_block) = 1
#   - one-hot equality: s_block^T (11^T - I) s_block = 0
reg4 = ConstraintRegistry()
for sl, levels in zip(slices, blocks):
    reg4.add_block(idx=range(sl.start, sl.stop), levels=levels, sum_to_one=True, one_hot=True)

# ---- init: uniform within each block ----
x0 = np.zeros(n)
for sl in slices:
    x0[sl] = 1.0 / (sl.stop - sl.start)

# ---- ALM configuration ----
cfg4 = ALMConfig(max_outer=30, tol_h=1e-4, tol_g=1e-6, rho_h=30.0, rho_g=30.0)

# ---- Dirac-3 inner solver kwargs with GLOBAL DEVICE SUM = number of blocks ----
num_blocks = len(blocks)
solver_kwargs = {
    "num_samples": 10,
    "relaxation_schedule": 2,
    "sum_constraint": num_blocks,
}

# ---- run ----
res4 = ALMAlgorithm.run(model4, reg4, Dirac3ContinuousCloudSolver(), cfg4, x0=x0, verbose=True, **solver_kwargs)

x4 = res4["x"]
print("\nFinal block selections")
for bi, (sl, levels) in enumerate(zip(slices, blocks)):
    s = x4[sl]
    j = int(np.argmax(s))
    print(f"  Block {bi}: s = {np.round(s,4)} -> chosen index {j}, value {levels[j]}")
print("Decoded:", res4["decoded"])

2025-11-25 10:06:43 - Dirac allocation balance = 0 s (unmetered)
2025-11-25 10:06:44 - Job submitted: job_id='6925c60408442f441bb69f3e'
2025-11-25 10:06:44 - QUEUED
2025-11-25 10:06:46 - RUNNING
2025-11-25 10:08:50 - COMPLETED
2025-11-25 10:08:53 - Dirac allocation balance = 0 s (unmetered)
[ALM 00] f=-2 | eq_inf=8.21e-02 | ineq_inf=0.00e+00 | rho_h=3.00e+01 | rho_g=3.00e+01
2025-11-25 10:08:55 - Dirac allocation balance = 0 s (unmetered)
2025-11-25 10:08:55 - Job submitted: job_id='6925c68708442f441bb69f3f'
2025-11-25 10:08:55 - QUEUED
2025-11-25 10:08:58 - RUNNING
2025-11-25 10:09:01 - COMPLETED
2025-11-25 10:09:03 - Dirac allocation balance = 0 s (unmetered)
[ALM 01] f=-2 | eq_inf=8.21e-02 | ineq_inf=0.00e+00 | rho_h=3.00e+01 | rho_g=3.00e+01
2025-11-25 10:09:05 - Dirac allocation balance = 0 s (unmetered)
2025-11-25 10:09:06 - Job submitted: job_id='6925c69208442f441bb69f40'
2025-11-25 10:09:06 - QUEUED
2025-11-25 10:09:08 - RUNNING
2025-11-25 10:09:14 - COMPLETED
2025-11-25 10:09: